In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
import numpy as np

In [2]:
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,6000);")
    time.sleep(2)

In [3]:
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('C:\chromedriver.exe')

# 키워드를 검색한 유튜브 주소 접속
#keyword = '후드'

#url = 'https://www.youtube.com/results?search_query={}'.format(keyword)
url = 'https://www.youtube.com/c/%ED%95%8F%EB%8D%94%EC%82%AC%EC%9D%B4%EC%A6%88/videos'
driver.get(url)

#driver.maximize_window()

In [4]:
# 제목 & 링크 크롤링

# 스크롤 다운 10번 실행
body = driver.find_element_by_tag_name('body')
body.send_keys(Keys.PAGE_DOWN)

for i in range(1,10):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)

soup = BeautifulSoup(driver.page_source, 'html.parser')
name = soup.select('a#video-title')
video_url = soup.select('a#video-title')

video_num_list = [] #영상번호
name_list = [] #영상제목
url_list = [] #영상주소

for i in range(len(name)):
    video_num_list.append(i+1)
    name_list.append(name[i].text.strip())

for i in video_url:
    url_list.append('{}{}'.format('https://www.youtube.com',i.get('href')))
    
youtubeDic = {
    '영상 번호' : video_num_list,
    '제목': name_list,
    '주소': url_list,
}

youtubeDf = pd.DataFrame(youtubeDic)
youtubeDf.head()

,영상 번호,제목,주소
0,1,남자 트렌디한 헤어스타일 추천과 여름 필수 관리법👀,https://www.youtube.com/watch?v=z1jOb9PTe9E
1,2,여름 쇼핑 드가자😍 고민 형이 대신해줄게 카드만 줘봐,https://www.youtube.com/watch?v=ayCpzxIRUD0
2,3,[남자 여름 코디] 브랜드 5개中 읏되는 거 딱 1개씩만 추천함ㅇㅇ,https://www.youtube.com/watch?v=yYGFxvSCnAc
3,4,남자 여름 코디&쇼핑 추천☀ 맞춤으로 간다,https://www.youtube.com/watch?v=PEcOlVZd8wQ
4,5,22년 인기 있는 신발 총 집합👟,https://www.youtube.com/watch?v=cKzuoJLUTU4


In [5]:
youtubeDf = youtubeDf.iloc[0:5]

In [7]:
youtubeDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   영상 번호   5 non-null      int64 
 1   제목      5 non-null      object
 2   주소      5 non-null      object
dtypes: int64(1), object(2)
memory usage: 248.0+ bytes


In [10]:
name_list = name_list[0:5]
url_list = url_list[0:5]

In [11]:
# 댓글 크롤링
video_num = 0
video_idex=[]
video_comm = []


for i in range(len(name_list)):
    driver.get(url_list[i])
    
    time.sleep(3)
    
    try :
        # 동영상 재생 멈추기 
        body = driver.find_element_by_tag_name('body')
        body.send_keys(Keys.SPACE)
    except: # 스페이스 누르기 안되면 직접 일시중지 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[1]/div/div/div/ytd-player/div/div/div[1]/video').click()
        
    # 유튜브 프리미엄 팝업창 닫기
    try:
        driver.find_element_by_css_selector("#dismiss-button > a").click()
    except:
        pass

    # 댓글 보이게 하기(밑으로 내리기)
    time.sleep(1)
    driver.execute_script("window.scrollTo(0,800);")
    time.sleep(8)
    
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(3)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(5)
    
    
    # 댓글 더보기 클릭 (대댓글)
    # buttons = driver.find_elements_by_css_selector("#more-replies > a")
    # time.sleep(1)
    # for button in buttons:
    #     button.send_keys(Keys.ENTER)
    #     time.sleep(1)
    #     button.send_keys(Keys.ENTER)
        
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    comment_list = soup.select("yt-formatted-string#content-text")

    comment_final = []

    for i in range(len(comment_list)):
        temp_comment = comment_list[i].text
        temp_comment = temp_comment.replace('\n', '')
        temp_comment = temp_comment.replace('\t', '')
        temp_comment = temp_comment.replace('    ', '')
        temp_comment = temp_comment.replace('\xa0@', '')
        temp_comment = temp_comment.replace('\xa0', '')
        temp_comment = temp_comment.replace('\r', '')
        comment_final.append(temp_comment)
    
    video_idex.append(video_num)
    video_comm.append(comment_final)
    video_num += 1


In [23]:
df = []
for i in range(len(video_idex)):
    if len(video_comm[i]) != 0:
        n = np.full((len(video_comm[i])),i+1)
        temp = pd.DataFrame([ x for x in zip(n,video_comm[i])])
        df.append(temp)

In [24]:
comment_df = df[0]
for i in range(1, len(df)):
    comment_df = pd.concat([comment_df, df[i]])
    
comment_df.columns = ['영상 번호', '댓글']
comment_df.head()

,영상 번호,댓글
0,1,이런 컨텐츠 좋은 거 같아요~몰랐던 브래드도 알고 메인템도 알아서 좋네요~
1,1,각 브랜드에서 매력있는 아이템들이 많은걸 알아서 좋았네요 ㅎㅎ
2,1,형 이런 컨셉 유료광고면 한달에 3번 이상 유료광고 받아도 볼만한 느낌… 짱이다 형️
3,1,마지막 바지 예뻐서 사려고봤더니 품절이네요 ㅠㅠ 재입고되면 바로사야지 !
4,1,와 데님셔츠는 가격대비 미치긴한듯


In [25]:
comment_df['영상 번호'].nunique()

103

In [11]:
#CSV형태로 저장하기
youtubeDf.to_csv('./'+'유튜브_핏더사이즈_크롤링.csv', encoding='utf-8-sig', index=False)
comment_df.to_csv('./'+'유튜브_핏더사이즈_댓글.csv', encoding='utf-8-sig', index=False)